## Подбор гиперпараметров

In [1]:
from sklearn.model_selection import StratifiedKFold
import optuna
import json
from optuna.visualization import plot_optimization_history, plot_param_importances
from catboost import CatBoostClassifier, Pool
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from notebooks.helpers import explore_data_modern
from catboost.utils import get_gpu_device_count
from sklearn.metrics import roc_auc_score
from datetime import datetime

In [2]:
print(f"Доступно GPU: {get_gpu_device_count()}")

Доступно GPU: 1


Настройки отображения

In [3]:
pd.set_option('display.max_columns', 50)
plt.style.use('ggplot') 

Загрузка данных

In [4]:
try:
    X_CROSS = pd.read_parquet('../data/datasets/cross.parquet')
            
    print("✅ Данные загружены!")
    print(f"Cross validation frame: {X_CROSS.shape[0]} строк")
except Exception as e:
    print(f"❌ Ошибка: {e}")

✅ Данные загружены!
Cross validation frame: 1385812 строк


In [5]:
display(explore_data_modern(X_CROSS, 'Train'))


🔍 Анализ датафрейма: Train


In [10]:
def create_objective(feature_list, cat_features):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    def objective(trial):
        # 2. Подбираем гиперпараметры
        params = {
            'depth': trial.suggest_int('depth', 4, 8),
            'learning_rate': trial.suggest_float('lr', 0.01, 0.3, log=True),
            'iterations': trial.suggest_int('iterations', 300, 1000),
            'l2_leaf_reg': trial.suggest_int('l2', 1, 5),
            'task_type': 'GPU',  # Включаем GPU
            'devices': '0',    # Используем первую видеокарту (или '0' для одной GPU)
            'early_stopping_rounds': 50,  # Ранняя остановка для экономии времени
            # 'verbose': False,
            'border_count': 128
        }
        
        # 3. Оценка модели
        model = CatBoostClassifier(**params, silent=True)
        scores = []
        for train_idx, val_idx in skf.split(X_CROSS[feature_list], X_CROSS['target']):
            X_train_fold, X_val_fold = X_CROSS.iloc[train_idx][feature_list], X_CROSS.iloc[val_idx][feature_list]
            y_train_fold, y_val_fold = X_CROSS['target'].iloc[train_idx], X_CROSS['target'].iloc[val_idx]
        
            train_pool = Pool(X_train_fold, y_train_fold, cat_features=cat_features)
            val_pool = Pool(X_val_fold, y_val_fold, cat_features=cat_features)
        
            model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)
            scores.append(roc_auc_score(y_val_fold, model.predict_proba(X_val_fold)[:, 1]))
        return np.mean(scores)
    return objective

**Вот и настройка**

In [23]:
def run_study(external_features, cat_features, results_dir='../data/models/hyper'):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    json_filename = f"best_params_{timestamp}.json"
    csv_filename = f"optimization_results_{timestamp}.csv"
    json_path = f"{results_dir}/{json_filename}"
    csv_path = f"{results_dir}/{csv_filename}"
        
    study = optuna.create_study(direction='maximize')
    objective_func = create_objective(external_features, cat_features)
    study.optimize(objective_func, n_trials=50)
    # study.optimize(objective_func, n_trials=2)
    
    fig1 = plot_optimization_history(study)
    fig2 = plot_param_importances(study)
    
    fig1.show()
    fig2.show()
    
    results_df = study.trials_dataframe()
    results_df.to_csv(csv_path, index=False)
    
    print("Все параметры из study.best_params:", study.best_params.keys())
    
    best_params = {
        'depth': study.best_params['depth'],
        'iterations': study.best_params['iterations'],
        'l2_leaf_reg': study.best_params['l2'],
        'learning_rate': study.best_params['lr'],
        'roc_auc': study.best_value,
        'timestamp': timestamp,
        'features_used': external_features,
        'categorical_features': cat_features
    }
    
    with open(json_path, 'w') as f:
        json.dump(best_params, f, indent=4)
    
    print(f"\nРезультаты сохранены в:")
    print(f"- Параметры модели: {json_path}")
    print(f"- Полные результаты: {csv_path}")
    
    print("\nЛучшие параметры:")
    for key, value in study.best_params.items():
        print(f"{key}: {value}")
    print(f"\nЛучший ROC-AUC: {study.best_value:.4f}")
    
    best_trial = study.best_trial
    
    print("\nЗначимость фичей из лучшей модели:")
    best_model = CatBoostClassifier(
        **{k: v for k, v in best_trial.params.items() 
           if k in ['depth', 'learning_rate', 'iterations', 'l2_leaf_reg']},
        cat_features=cat_features,
        verbose=False
    )
    best_model.fit(X_CROSS[external_features], X_CROSS['target'])
    
    feature_importances = pd.DataFrame({
        'feature': external_features,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(feature_importances.to_string(index=False))
    return study

In [24]:
param_features = ['utm_source', 'utm_medium', 'device_brand', 'visit_number', 'utm_campaign', 'utm_keyword']
category_features = ['utm_source', 'utm_medium', 'device_brand', 'utm_campaign', 'utm_keyword']

In [25]:
run_study(param_features, category_features)

[I 2025-07-11 22:50:00,421] A new study created in memory with name: no-name-2be3b8e9-537a-466f-8687-27d49199d79f
[I 2025-07-11 22:54:23,197] Trial 0 finished with value: 0.709644866540396 and parameters: {'depth': 4, 'lr': 0.05304142258749048, 'iterations': 818, 'l2': 1}. Best is trial 0 with value: 0.709644866540396.
[I 2025-07-11 22:59:30,861] Trial 1 finished with value: 0.7095170831503549 and parameters: {'depth': 6, 'lr': 0.01907892941919377, 'iterations': 716, 'l2': 1}. Best is trial 0 with value: 0.709644866540396.
[I 2025-07-11 23:03:07,112] Trial 2 finished with value: 0.7104623096538358 and parameters: {'depth': 5, 'lr': 0.12255001524651231, 'iterations': 699, 'l2': 5}. Best is trial 2 with value: 0.7104623096538358.
[I 2025-07-11 23:08:24,921] Trial 3 finished with value: 0.7101190945958815 and parameters: {'depth': 4, 'lr': 0.08434674680543174, 'iterations': 945, 'l2': 1}. Best is trial 2 with value: 0.7104623096538358.
[I 2025-07-11 23:12:24,132] Trial 4 finished with val

Все параметры из study.best_params: dict_keys(['depth', 'lr', 'iterations', 'l2'])

Результаты сохранены в:
- Параметры модели: ../data/models/hyper/best_params_20250711_225000.json
- Полные результаты: ../data/models/hyper/optimization_results_20250711_225000.csv

Лучшие параметры:
depth: 8
lr: 0.08743604703974947
iterations: 672
l2: 4

Лучший ROC-AUC: 0.7118

Значимость фичей из лучшей модели:
     feature  importance
  utm_medium   29.068976
  utm_source   21.772095
utm_campaign   16.348888
device_brand   14.083492
visit_number   11.415968
 utm_keyword    7.310581


In [26]:
del X_CROSS